In [1]:
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models import CategoricalColorMapper, Legend
from bokeh.palettes import Category10
from bokeh.layouts import layout, column, row
from bokeh.models import ColumnDataSource, GeoJSONDataSource, Slider, Button, Patches, Select, Range1d, CategoricalColorMapper
from bokeh.plotting import figure
from bokeh.io import show, output_notebook, curdoc

from data import get_merged_data

output_notebook()

Loading BokehJS ...

In [7]:
Category10.keys()

dict_keys([3, 4, 5, 6, 7, 8, 9, 10])

In [88]:
df = get_merged_data(year_as_datetime=False)
df.head()

indicator         completion_rate                                            \
level                     primary                      secondary              
gender                     female      male     total     female       male   
country_code year                                                             
ABW          2000        89.06955  91.75029  90.32299  52.930729  59.478802   
AFG          2000         0.85208   7.99266   4.48156   0.740330   7.447390   
AGO          2000             NaN       NaN       NaN        NaN        NaN   
ALB          2000             NaN       NaN       NaN        NaN        NaN   
AND          2000             NaN       NaN       NaN        NaN        NaN   

indicator                                                    \
level                         tertiary                        
gender                 total    female      male      total   
country_code year                                             
ABW          2000  55.992432  14.74587  18.27305  16.395081   
AFG          2000   4.149460       NaN       NaN        NaN   
AGO          2000        NaN       NaN       NaN        NaN   
ALB          2000        NaN       NaN       NaN        NaN   
AND          2000        NaN       NaN       NaN        NaN   

indicator         compulsory_education_duration  ... learning_outcome  \
level                                     total  ...          primary   
gender                                    total  ...            total   
country_code year                                ...                    
ABW          2000                           NaN  ...              NaN   
AFG          2000                           6.0  ...              NaN   
AGO          2000                           4.0  ...              NaN   
ALB          2000                           8.0  ...              NaN   
AND          2000                          10.0  ...              NaN   

indicator                                                                      \
level               secondary                               total               
gender                 female        male       total      female        male   
country_code year                                                               
ABW          2000         NaN         NaN         NaN         NaN         NaN   
AFG          2000         NaN         NaN         NaN         NaN         NaN   
AGO          2000         NaN         NaN         NaN         NaN         NaN   
ALB          2000  406.148583  375.039887  390.869437  406.148583  375.039887   
AND          2000         NaN         NaN         NaN         NaN         NaN   

indicator                          gdppc    population education_spent  
level                              total         total           total  
gender                  total      total         total           total  
country_code year                                                       
ABW          2000         NaN        NaN           NaN             NaN  
AFG          2000         NaN   502.3727  22461.000000       21.970746  
AGO          2000         NaN  2013.6362  11676.399620       52.506170  
ALB          2000  390.869437  4808.4796   3164.048845      164.939028  
AND          2000         NaN        NaN           NaN             NaN  

[5 rows x 91 columns]

In [89]:
def select_range(indicator, level='total', gender='total', log=False):
    min = df[(indicator, level, gender)].min() * 0.9
    max = df[(indicator, level, gender)].max() * 1.1
    
    return min, max

In [90]:
df[('gdppc', 'total', 'total')].min()
df[('gdppc', 'total', 'total')].max()

156299.0

In [91]:
def app(doc):
    DATES = sorted(df.index.get_level_values('year').unique())
    REGIONS = sorted(df[('region', 'total', 'total')].unique())
    INCOME_GROUPS = sorted(df[('income_group', 'total', 'total')].unique())
    
    source = ColumnDataSource(df.xs(DATES[-1], level='year').xs(('total', 'total'), axis=1, level=('level', 'gender')))

    # color_mapper = CategoricalColorMapper(palette=COLOR_PALETTE)
    
    def update_year(attr, old, new):
        source.data = df.xs(new, level='year').xs(('total', 'total'), axis=1, level=('level', 'gender'))
    
    def update_color(attr, old, new):
        pass

    # year slider
    slider_year = Slider(
        title='Year',
        value=DATES[-1],
        start=DATES[0],
        end=DATES[-1],
        step=1)
    slider_year.on_change('value', update_year)
    
    select_color = Select(
        title='Color by:',
        value='region',
        options=['region', 'income_group'])
    select_color.on_change('value', update_color)

    def scatter():
        p = figure(height=400, width=800,
                title = 'Harmonized Learning outcome by GDPPC',
                x_axis_type='log',
                y_range=select_range('learning_outcome'),
                x_range=select_range('gdppc'))
        p.xaxis.axis_label = 'log GDP per capita'
        p.yaxis.axis_label = 'Harmonized Learning Outcome'
        p.add_layout(Legend(), 'right')
        p.legend.location = 'top_left'
        p.legend.title = 'Regions'

        p.scatter('gdppc', 'learning_outcome',
                source=source,
                color=factor_cmap(select_color.value, 'Category10_7', REGIONS),
                size=9,
                legend_group='region',
                fill_alpha=0.5)
        return p
    
    tools = column(slider_year, select_color)
    top_right = column(scatter())
    layout = row(tools, top_right)
    
    # Add to server
    doc.add_root(layout)

In [92]:
show(app)

ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'vscode-webview://1b26404d-6d1e-4eb2-a3d0-75058ab3dc91';                       use --allow-websocket-origin=1b26404d-6d1e-4eb2-a3d0-75058ab3dc91 or set BOKEH_ALLOW_WS_ORIGIN=1b26404d-6d1e-4eb2-a3d0-75058ab3dc91 to permit this; currently we allow origins {'localhost:8888'}
ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'vscode-webview://1b26404d-6d1e-4eb2-a3d0-75058ab3dc91';                       use --allow-websocket-origin=1b26404d-6d1e-4eb2-a3d0-75058ab3dc91 or set BOKEH_ALLOW_WS_ORIGIN=1b26404d-6d1e-4eb2-a3d0-75058ab3dc91 to permit this; currently we allow origins {'localhost:8888'}
